The challenge is as follows:

```py
from Crypto.Cipher import AES
from secret import key, FLAG

p = 4170887899225220949299992515778389605737976266979828742347
ct = bytes.fromhex("ae7d2e82a804a5a2dcbc5d5622c94b3e14f8c5a752a51326e42cda6d8efa4696")

def crack_safe(key):
    return pow(7, int.from_bytes(key, 'big'), p) == 0x49545b7d5204bd639e299bc265ca987fb4b949c461b33759

assert crack_safe(key) and AES.new(key,AES.MODE_ECB).decrypt(ct) == FLAG
```

Note that we must have that $$ 7 ^ k = \mathtt{0x495}\cdots\mathtt{759} \pmod{p} $$ where $p$ is a 192 bit prime.

While this is already not secure, we have that $\phi(p) = p-1$ has many small factors:

```gp
? p=4170887899225220949299992515778389605737976266979828742347;factorint(p-1)
%1 = 
[                        2 1]

[                       19 1]

[                      151 1]

[                      577 1]

[                    67061 1]

[              18279232319 1]

[             111543376699 1]

[9213409941746658353293481 1]
```

For the smaller factors, we can use Sage's built-in baby-step giant-step implementation which runs in $O(\sqrt{p})$; however, for the largest prime factor $p'$, we use CADO-NFS to calculate this part of the discrete logarithm:

```
$ ./cado-nfs.py -dlp -ell 9213409941746658353293481 target=1798034623618994974454756356126246972179657041628028417881 4170887899225220949299992515778389605737976266979828742347 -t all --workdir /tmp/cado-nfs-tmp
[[...]]
Info:root: If you want to compute one or several new target(s), run ./cado-nfs.py /tmp/cado-nfs-tmp/p60.parameters_snapshot.3 target=<target>[,<target>,...]
Info:root: logbase = 689700230313623370222183478814904246546188182712829892313
Info:root: target = 1798034623618994974454756356126246972179657041628028417881
Info:root: log(target) = 2215765705042274080663116 mod ell
2215765705042274080663116
$ ./cado-nfs.py /tmp/cado-nfs-tmp/p60.parameters_snapshot.0 target=7
[[...]]
Info:root: logbase = 689700230313623370222183478814904246546188182712829892313
Info:root: target = 7
Info:root: log(target) = 6424341129540508417798214 mod ell
6424341129540508417798214

```

Therefore, we have:

$$ k \equiv \frac{2215765705042274080663116}{6424341129540508417798214} \pmod{p'}$$

Now, use Pohlig-Hellman and Sage to reconstruct the original discrete logarithm:

In [66]:
from sage.all import *
from sage.groups.generic import bsgs

p = 4170887899225220949299992515778389605737976266979828742347
g = Mod(7, p)
h = Mod(1798034623618994974454756356126246972179657041628028417881, p)

p_large = 9213409941746658353293481

e_precalc = int(Integers(p_large)(2215765705042274080663116)/(6424341129540508417798214))

exponents, moduli = [], []

for p_, e_ in list(factor(p-1)):
    # print(p_, e_)
    g_i = pow(g, (p-1)//(p_**e_))
    h_i = pow(h, (p-1)//(p_**e_))
    if p_ != p_large:
        try:
            exponents.append(bsgs(g_i, h_i, (0, p_**e_-1)))
            moduli.append(p_**e_)
        except:
            continue
    else:
        exponents.append(e_precalc)
        moduli.append(p_large)
        
k = int(crt(exponents, moduli))
assert pow(g, k, p) == h

We can now decrypt the flag:

In [67]:
from Crypto.Cipher import AES

ct = bytes.fromhex("ae7d2e82a804a5a2dcbc5d5622c94b3e14f8c5a752a51326e42cda6d8efa4696")
AES.new(int.to_bytes(k, 128//8, 'big'), AES.MODE_ECB).decrypt(ct)

b'uiuctf{Dl0g_w/__UnS4F3__pR1Me5_}'